In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pickle


# Predict with fixed number of fields
'time','REPORT_ID','DISTANCE_IN_METERS','vehicleCount',vehicleCount_lag_1 ,vehicleCount_lag_1

In [77]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [78]:
class MyDecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree '''
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    # update the best split if needed
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def variance_reduction(self, parent, l_child, r_child):
        ''' function to compute variance reduction '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        print('X: ', X)
        print('Y: ', Y)
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [79]:
# load the model
with open('../model/MyDecisionTreeRegressorModelWithDOW.sav', 'rb') as f:
    model = pickle.load(f)

In [80]:
#import mysql connector
import sqlalchemy
#connect to mysql
database_username = 'namnp'
database_password = '12345678'
database_ip       = '127.0.0.1'
database_name     = 'iot'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [81]:
# load data from mysql
df = pd.read_sql('SELECT * FROM `traffic_with_lag_data` order by REPORT_ID, timestamp', con=database_connection)
df.head(3)

,time,REPORT_ID,DISTANCE_IN_METERS,vehicleCount,timestamp,day_of_week,vehicleCount_lag_1,vehicleCount_lag_2
0,9,158324,1030,75,2014-08-01 09:00:00,4,78.0,11.0
1,10,158324,1030,50,2014-08-01 10:00:00,4,75.0,78.0
2,11,158324,1030,100,2014-08-01 11:00:00,4,50.0,75.0


In [82]:
#get hour from timestamp
# df['time'] = df['timestamp'].apply(lambda x: x.split(' ')[1].split(':')[0])
# df['time'] = df['time'].astype(int)
# ['time','day_of_week','REPORT_ID','DISTANCE_IN_METERS','vehicleCount_lag_1','vehicleCount_lag_2','vehicleCount']
cols = ['time','day_of_week','REPORT_ID','DISTANCE_IN_METERS','vehicleCount_lag_1','vehicleCount_lag_2','vehicleCount']
df = df[cols]
df.head(3)

,time,day_of_week,REPORT_ID,DISTANCE_IN_METERS,vehicleCount_lag_1,vehicleCount_lag_2,vehicleCount
0,9,4,158324,1030,78.0,11.0,75
1,10,4,158324,1030,75.0,78.0,50
2,11,4,158324,1030,50.0,75.0,100


In [83]:
input = df[(df['time']==6) & (df['REPORT_ID']==158895)]
input_data = input.iloc[:, :-1].values
input_data

array([[6.00000e+00, 5.00000e+00, 1.58895e+05, 1.50500e+03, 5.00000e+00,
        9.00000e+00],
       [6.00000e+00, 6.00000e+00, 1.58895e+05, 1.50500e+03, 8.00000e+00,
        5.00000e+00],
       [6.00000e+00, 0.00000e+00, 1.58895e+05, 1.50500e+03, 1.80000e+02,
        9.80000e+01],
       [6.00000e+00, 1.00000e+00, 1.58895e+05, 1.50500e+03, 1.35000e+02,
        9.90000e+01],
       [6.00000e+00, 2.00000e+00, 1.58895e+05, 1.50500e+03, 1.68000e+02,
        1.31000e+02],
       [6.00000e+00, 3.00000e+00, 1.58895e+05, 1.50500e+03, 1.33000e+02,
        8.20000e+01],
       [6.00000e+00, 4.00000e+00, 1.58895e+05, 1.50500e+03, 1.27000e+02,
        1.11000e+02],
       [6.00000e+00, 5.00000e+00, 1.58895e+05, 1.50500e+03, 1.20000e+01,
        3.00000e+00],
       [6.00000e+00, 6.00000e+00, 1.58895e+05, 1.50500e+03, 5.00000e+00,
        9.00000e+00],
       [6.00000e+00, 0.00000e+00, 1.58895e+05, 1.50500e+03, 1.66000e+02,
        1.23000e+02],
       [6.00000e+00, 1.00000e+00, 1.58895e+05, 1.5

In [84]:
result = model.predict(input_data)
result

[7.71890625,
 7.71890625,
 149.11952429331262,
 100.51245390663048,
 149.11952429331262,
 100.51245390663048,
 100.51245390663048,
 17.025015315499285,
 7.71890625,
 149.11952429331262,
 149.11952429331262,
 17.025015315499285,
 63.97860905371802,
 149.11952429331262,
 7.71890625,
 17.025015315499285,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 7.71890625,
 7.71890625,
 149.11952429331262,
 243.40103412616338,
 149.11952429331262,
 243.40103412616338,
 149.11952429331262,
 17.025015315499285,
 7.71890625,
 149.11952429331262,
 243.40103412616338,
 243.40103412616338,
 149.11952429331262,
 100.51245390663048,
 17.025015315499285,
 17.025015315499285,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 100.51245390663048,
 149.11952429331262,
 17.025015315499285,
 7.71890625,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 149.11952429331262,
 17.025015315499285,
 7.71890625,
 1

In [85]:
from sklearn.metrics import r2_score    
np.sqrt(r2_score(input['vehicleCount'].values, result))

0.891495911888917

In [86]:
df_result = pd.DataFrame({'actual': input['vehicleCount'].values, 'predict': result})
df_result.head(20)

,actual,predict
0,21,7.718906
1,7,7.718906
2,117,149.119524
3,112,100.512454
4,105,149.119524
5,105,100.512454
6,101,100.512454
7,15,17.025015
8,14,7.718906
9,148,149.119524
